# Proglearn: Scene Segmentation
*Neuro Data Design I: Fall 2021*


This tutorial provides a walkthrough into applying Proglearn to perform scene segmentation on the ADE20K image dataset.


**Contributors**
- Kevin Rao (krao15@jhu.edu)
- Amy van Ee (avanee1@jhu.edu)
- Narayani Wagle (nwagle1@jhu.edu)

# I. Preprocessing of Images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
# code

# II. Other Methods of Scene Segmentation

## II.A. Random Forests

To test other methods of scene segmentation before implementation of Python, we will use scikit-image. 

**Visualize an Example Image**

We will choose the image of a palm tree on a beach from the ADE20K dataset to confirm that this method works.

In [ ]:
#========================================================#
# Plot the original image alongside the annotated image

fig, ax = plt.subplots(1,2)
fig.set_size_inches(10,10)

image = cv2.imread(imgpaths[9])
label = cv2.imread(lblpaths[9]) 
ax[0].imshow(image)
ax[0].set_title("Original Image")

ax[1].imshow(label)
ax[1].set_title("True Annotations")
plt.show()

In [ ]:
#========================================================#
# Show a histogram of the image

# tuple to select colors of each channel line
colors = ("red", "green", "blue")
channel_ids = (0, 1, 2)

# create the histogram plot, with three lines, one for
# each color
plt.xlim([0, 256])
for channel_id, c in zip(channel_ids, colors):
    histogram, bin_edges = np.histogram(
        label[:, :, channel_id], bins=256, range=(0, 256)
    )
    plt.plot(bin_edges[0:-1], histogram, color=c)

plt.xlabel("Color Value")
plt.ylabel("Pixel Count")
plt.title("Histogram of Annotated Images in RGB: %d labels" %(np.unique(label).size))

plt.show()

**Visualize the Greyscale of the Image**

Although the annotated image from the ADE20K dataset has three color channels (RGB) as just shown in the histogram, scikit-image produces an annotated image that has only one. Therefore, in using the provided functions in this package, we will convert to greyscale.

In [ ]:
#========================================================#
# Convert to greyscale

from skimage.color import rgb2gray

image_grey = rgb2gray(image)
label_grey = rgb2gray(label)
label_grey = label_grey * 1000
label_grey = label_grey.astype(int)

fig, ax = plt.subplots(1,2)
fig.set_size_inches(10,10)

ax[0].imshow(image_grey, cmap = plt.cm.gray)
ax[0].set_title("Original Image")

ax[1].imshow(label_grey, cmap = plt.cm.gray)
ax[1].set_title("True Annotations")
plt.show()

In [ ]:
#========================================================#
# Show a histogram of the grey-scale image

plt.hist(label_grey)
plt.title("Histogram of Greyscale Annotated Image: %d Classes" %(np.unique(label_grey).size))
plt.xlabel("Intensity")
plt.ylabel("Pixel Count")

**Performing the Image Segmentation**

In [ ]:
#========================================================#
# Use scikit-image to perform Image Segmentation

from skimage import segmentation, feature, future
from functools import partial

img = image_grey
training_labels = label_grey

sigma_min = 1
sigma_max = 16
features_func = partial(feature.multiscale_basic_features,
                        intensity=True, edges=False, texture=True,
                        sigma_min=sigma_min, sigma_max=sigma_max,
                        multichannel=True)
features = features_func(img)
clf = RandomForestClassifier(n_estimators=50, n_jobs=-1,
                             max_depth=10, max_samples=0.05)
clf = future.fit_segmenter(training_labels, features, clf)
result = future.predict_segmenter(features, clf)

fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(9, 4))
ax[0].imshow(segmentation.mark_boundaries(img, result, mode='thick'))
ax[0].contour(training_labels)
ax[0].set_title('Image, mask and segmentation boundaries')
ax[1].imshow(result, cmap = plt.cm.gray)
ax[1].set_title('Segmentation')
fig.tight_layout()

**Analyzing Accuracy**

We will next analyze the performance of scikit-image by analyzing the accuracy. We will do so by comparing the outcome from scene segmentation and the true annotated image.

In [ ]:
#========================================================#
# Analyze the accuracy by looking at 
# precision, recall, false splits, false merges, dice score

from skimage.metrics import (adapted_rand_error,
                              variation_of_information)
def get_dice(true, test):
    return np.size(test[test == true]) * 2.0 / (np.size(true) + np.size(test))

error, precision, recall = adapted_rand_error(label_grey, result)
splits, merges = variation_of_information(label_grey, result)
dice = get_dice(label_grey, result)
print(f'Adapted Rand error: {error}')
print(f'Adapted Rand precision: {precision}')
print(f'Adapted Rand recall: {recall}')
print(f'False Splits: {splits}')
print(f'False Merges: {merges}')
prift(f'Dice Coefficient: {dice}')

fig, axes = plt.subplots(2, 1, figsize=(6, 6), constrained_layout=True)
ax = axes.ravel()

ax[0].scatter(merges, splits)
ax[0].set_xlabel('False Merges (bits)')
ax[0].set_ylabel('False Splits (bits)')
ax[0].set_title('Split Variation of Information')

ax[1].scatter(precision, recall)
ax[1].set_xlabel('Precision')
ax[1].set_ylabel('Recall')
ax[1].set_title('Adapted Random precision vs. recall')
ax[1].set_xlim(0, 1)
ax[1].set_ylim(0, 1)

plt.show()

Evidently, it appears that scikit-image did a relatively good job of scene segmentation. Thus, we proceed to apply it to a larger dataset.

**Performing Scene Segmentation on entire dataset**

Next, we will try to perform scene segmentation on the entire dataset. However, it was found that the original images and annotated images did not have the same dimensionality, which was a requirement for scikit-image's functionality. Therefore, we selectively chose images whose original and annotated versions had the same dimensions. 

In [ ]:
#========================================================#
# convert all images to greyscale
images_grey = np.array([rgb2gray(cv2.imread(imgpath)) for imgpath in imgpaths])
labels_grey = np.array([(rgb2gray(cv2.imread(lblpath))*1000).astype(int) for lblpath in lblpaths])

In [ ]:
#========================================================#
# identify images with matching dimensions between image and label
images_grey_match = images_grey[[images_grey[i].shape == labels_grey[i].shape for i in np.arange(len(images_grey))]]
labels_grey_match = labels_grey[[images_grey[i].shape == labels_grey[i].shape for i in np.arange(len(images_grey))]]

In [ ]:
#========================================================#
# perform scene segmentation on all images

# initialize arrays
error_list = np.zeros(len(images_grey_match))
precision_list = np.zeros(len(images_grey_match))
recall_list = np.zeros(len(images_grey_match))
splits_list = np.zeros(len(images_grey_match))
merges_list = np.zeros(len(images_grey_match))
dice_list = np.zeros(len(images_grey_match))
result_list = np.zeros(len(images_grey_match), dtype = object)

# loop through each image and determine values
for i in np.arange(len(images_grey_match)):
    # use classifier
    features = features_func(images_grey_match[i])
    result = future.predict_segmenter(features, clf)

    # assess
    error, precision, recall = adapted_rand_error(labels_grey_match[i], result)
    splits, merges = variation_of_information(labels_grey_match[i], result)
    dice = get_dice(labels_grey_match[i], result)

    # add to list
    error_list[i] = error
    precision_list[i] = precision
    recall_list[i] = recall
    splits_list[i] = splits
    merges_list[i] = merges
    dice_list[i] = dice
    result_list[i] = result

In [ ]:
#========================================================#
# analyze results

fig, axes = plt.subplots(3, 1, figsize=(6, 6), constrained_layout=True)
ax = axes.ravel()

ax[0].scatter(merges_list, splits_list)
ax[0].set_xlabel('False Merges (bits)')
ax[0].set_ylabel('False Splits (bits)')
ax[0].set_title('Split Variation of Information')

ax[1].scatter(precision_list, recall_list)
ax[1].set_xlabel('Precision')
ax[1].set_ylabel('Recall')
ax[1].set_title('Adapted Random precision vs. recall')
ax[1].set_xlim(0, 1)
ax[1].set_ylim(0, 1)

ax[2].hist(dice_list)
ax[2].set_xlabel('Dice Coefficient')
ax[2].set_ylabel('Frequency')
ax[2].set_title('Histogram of Dice Coefficients')

Evidently, it appears that scikit-image is only able to fit to one specific image at a time, and therefore only the image with the palm tree had high accuracy values, whereas the other images did not. 

**Looking at a Poorly Segmented Example**

In [ ]:
#========================================================#
# see low accuracy since had only trained on one image which has high dice coefficient

# original
fig, ax = plt.subplots(1,2)
fig.set_size_inches(10,10)

ax[0].imshow(images_grey_match[1], cmap = plt.cm.gray)
ax[0].set_title("Original Image")

ax[1].imshow(labels_grey_match[1], cmap = plt.cm.gray)
ax[1].set_title("True Annotations")
plt.show()

# segmented
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(9, 4))
ax[0].imshow(segmentation.mark_boundaries(images_grey_match[1], result_list[1], mode='thick'))
ax[0].contour(labels_grey_match[1])
ax[0].set_title('Image, mask and segmentation boundaries')
ax[1].imshow(result_list[1], cmap = plt.cm.gray)
ax[1].set_title('Segmentation')
fig.tight_layout()

This thus presents an opportunity for Proglearn to allow for improved and more generalizable scene segmentation.

## II.B. Neural Networks

In [ ]:
# code

# III. Proglearn for Scene Segmentation

## III.A. Random Forest

We next proceed to use the model of scikit-image to apply Proglearn for scene segmentation. We will use the LifelongClassificationForest in place of the RandomForestClassifier.

In [ ]:
#========================================================#
# perform a basic test using one task

from proglearn import LifelongClassificationForest

# define forest
progtree = LifelongClassificationForest()
progtree.add_task(images_grey_match[9], labels_grey_match[9])

# test prediction on a sample image
prediction = progtree.predict(images_grey_match[10], task_id = 0)
prediction_proba = progtree.predict_proba(images_grey_match[10], task_id = 0)

Unfortunately, in doing so we come across the error of dimensionality. Proglearn as it is implemented now only takes in a 1D array for the class label, whereas the dataset ADE20K consists of labels per pixel. Therefore, more work will need to be done to accommodate this specific dataset. An alternative approach will be to use another dataset where the labels are binary and therefore the problem is more simplified before we scale up to a more complex situation like the ADE20K dataset. 

Ultimately, we hope to be able to use Proglearn and test our scene segmentation implementation using more than one task. 

## III.B. Neural Networks